<a href="https://colab.research.google.com/github/mhakas/FinalProjectCC/blob/master/CCFinalProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colab for Predicting Titanic Survival


#### Import necessary packages

In [0]:
import numpy as np
from numpy import arange
from matplotlib import pyplot
import seaborn as sns
import pandas as pd
from pandas import read_csv
from pandas import set_option
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [60]:
!pip install sklearn.preprocessing
!pip install pandas


ERROR: Could not find a version that satisfies the requirement sklearn.preprocessing (from versions: none)
ERROR: No matching distribution found for sklearn.preprocessing


In [67]:
!pip install numpy

In [0]:
titanic = "https://raw.githubusercontent.com/mhakas/FinalProjectCC/master/train.csv"

In [0]:
df = pd.read_csv(titanic)

In [103]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### EDA

*   Survived: 0/1
*   PClass : ticket class
*   Sex: male/female (changed to 0/1 respectively)
*   SibSp : # Siblings/Spouses aboard Titanic
*   Parch: # Parents aboard Titanic
*   Fare: Continuous
*   Embarked: Port embarked from (C = Cherbourg, Q = Queenstown, S = Southampton



In [108]:

#df = df.drop(['PassengerId','Name','Ticket','Cabin'], axis = 1)
survival = df['Fare']
features = df.drop('Fare', axis = 1)
df.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [109]:
features.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,male,22.0,1,0,S
1,1,1,female,38.0,1,0,C
2,1,3,female,26.0,0,0,S
3,1,1,female,35.0,1,0,S
4,0,3,male,35.0,0,0,S


In [110]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [0]:
df['Sex']= pd.get_dummies(df['Sex'])
df['Embarked'] = pd.get_dummies(df['Embarked'])

In [0]:
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [0]:
df = df.dropna()

In [114]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### Modeling

In [115]:
survival = df['Fare']
#df = df.drop(['PassengerId','Name','Ticket','Cabin'], axis = 1)
features = df.drop('Fare', axis = 1)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [116]:
X = features
X.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Embarked
0,0,3,0,22.0,1,0,0
1,1,1,1,38.0,1,0,1
2,1,3,1,26.0,0,0,0
3,1,1,1,35.0,1,0,0
4,0,3,0,35.0,0,0,0


In [117]:
y = survival
y.head()

0     7.2500
1    71.2833
2     7.9250
3    53.1000
4     8.0500
Name: Fare, dtype: float64

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=22)

In [0]:
lr=LinearRegression()
lr.fit(X_train,y_train)

y_pred=lr.predict(X_test)

In [120]:
print('RMSE:', np.round(np.sqrt(metrics.mean_squared_error(y_test, lr.predict(X_test))), 2))
print('R2 score train:', np.round(r2_score(y_train, lr.predict(X_train), multioutput='variance_weighted'), 2)*100)
print('R2 score test:', np.round(r2_score(y_test, lr.predict(X_test), multioutput='variance_weighted'), 2))


RMSE: 43.32
R2 score train: 40.0
R2 score test: 0.36


In [121]:
y_train.describe()

count    712.000000
mean      32.016496
std       48.503690
min        0.000000
25%        7.895800
50%       14.479150
75%       31.275000
max      512.329200
Name: Fare, dtype: float64

#### Tune GBM

In [44]:
# Test options and evaluation metric using Root Mean Square error method
num_folds = 10
seed = 7
RMS = 'neg_mean_squared_error'
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
param_grid = dict(n_estimators=numpy.array([50,100,150,200,250,300,350,400]))
model = GradientBoostingRegressor(random_state=seed)
kfold = KFold(n_splits=num_folds, random_state=seed)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=RMS, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

NameError: ignored

In [0]:
# prepare the model
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)
model = GradientBoostingRegressor(random_state=seed, n_estimators=50)
model.fit(rescaledX, y_train)
# transform the validation dataset
rescaledValidationX = scaler.transform(X_test)
predictions = model.predict(rescaledValidationX)
print("Mean Squared Error")
print(mean_squared_error(y_test, predictions))
print("R2")
print(r2_score(y_test, predictions))

### Pickling linear model

In [0]:
from sklearn.externals import joblib


In [123]:
joblib.dump(lr, 'titanic_fare_prediction.joblib')


['titanic_fare_prediction.joblib']

In [124]:
!ls -l


total 12
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data
-rw-r--r-- 1 root root  677 May  3 21:55 titanic_fare_prediction.joblib
-rw-r--r-- 1 root root  941 May  3 21:12 titanic_prediction.joblib


In [0]:
clf = joblib.load('titanic_fare_prediction.joblib')


#### Adhoc Predict

In [127]:
actual_sample = df.head(5)
actual_sample

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,0
1,1,1,1,38.0,1,0,71.2833,1
2,1,3,1,26.0,0,0,7.9250,0
3,1,1,1,35.0,1,0,53.1000,0
4,0,3,0,35.0,0,0,8.0500,0


In [128]:
adhoc_predict = actual_sample[["Survived", "Pclass", "Sex", "Age", "SibSp", "Fare", "Embarked"]]
adhoc_predict.head()

,Survived,Pclass,Sex,Age,SibSp,Fare,Embarked
0,0,3,0,22.0,1,7.2500,0
1,1,1,1,38.0,1,71.2833,1
2,1,3,1,26.0,0,7.9250,0
3,1,1,1,35.0,1,53.1000,0
4,0,3,0,35.0,0,8.0500,0


In [129]:
json_payload = adhoc_predict.to_json()
json_payload

'{"Survived":{"0":0,"1":1,"2":1,"3":1,"4":0},"Pclass":{"0":3,"1":1,"2":3,"3":1,"4":3},"Sex":{"0":0,"1":1,"2":1,"3":1,"4":0},"Age":{"0":22.0,"1":38.0,"2":26.0,"3":35.0,"4":35.0},"SibSp":{"0":1,"1":1,"2":0,"3":1,"4":0},"Fare":{"0":7.25,"1":71.2833,"2":7.925,"3":53.1,"4":8.05},"Embarked":{"0":0,"1":1,"2":0,"3":0,"4":0}}'

In [130]:

scaler = StandardScaler().fit(adhoc_predict)
scaled_adhoc_predict = scaler.transform(adhoc_predict)
scaled_adhoc_predict

array([[-1.22474487,  0.81649658, -1.22474487, -1.50516598,  0.81649658,
        -0.81614059, -0.5       ],
       [ 0.81649658, -1.22474487,  0.81649658,  1.11251398,  0.81649658,
         1.53034706,  2.        ],
       [ 0.81649658,  0.81649658,  0.81649658, -0.85074599, -1.22474487,
        -0.79140535, -0.5       ],
       [ 0.81649658, -1.22474487,  0.81649658,  0.62169899,  0.81649658,
         0.86402362, -0.5       ],
       [-1.22474487,  0.81649658, -1.22474487,  0.62169899, -1.22474487,
        -0.78682475, -0.5       ]])

In [131]:
list(lr.predict(scaled_adhoc_predict))


[48.415002877468254,
 192.7789799104848,
 47.32690640178755,
 138.36595544460744,
 36.401749524274514]

In [132]:
list(clf.predict(scaled_adhoc_predict))


[48.415002877468254,
 192.7789799104848,
 47.32690640178755,
 138.36595544460744,
 36.401749524274514]

In [133]:
clf

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    uint8  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  891 non-null    uint8  
dtypes: float64(2), int64(4), uint8(2)
memory usage: 50.5 KB
